In [ ]:
!git clone -b alex "https://github.com/ab7289-tandon-nyu/csgy6953_DeepLearning_Midterm.git"
!cp -r /content/csgy6953_DeepLearning_Midterm/src/ .

In [ ]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt

import time
import copy
import random

In [ ]:
SEED = 1234

random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from src.data import get_transformed_data, make_data_loaders
from src.transforms import make_auto_transforms

BATCH_SIZE = 256
valid_ration = 0.1

train_iter, valid_iter, test_iter = (
    make_data_loaders(
        *get_transformed_data(
            make_transforms= make_auto_transforms,
            valid_ratio= valid_ration
        ),
        batch_size=BATCH_SIZE,
    )
)

In [ ]:
from src.model import ResNet, StemConfig, ResidualBlockType
from src.utils import initialize_parameters

arch = [
   (ResidualBlockType.BASIC, 1, 32, 0.1),
   (ResidualBlockType.BASIC, 4, 64, 0.1),
   (ResidualBlockType.BASIC, 7, 128, 0.1),
   (ResidualBlockType.BASIC, 4, 196, 0.1),
]

config = StemConfig(num_channels=32, kernel_size=3, stride=1, padding=1)
model = ResNet(arch, stem_config=config, output_size=10, use_bias=False)

In [ ]:
print("Initializing new model")
inputs = torch.ones((BATCH_SIZE, 3, 32, 32))
model = model.to(device)
y = model(inputs.to(device))
print(y.size())

model.apply(initialize_parameters)

In [ ]:
from fastai.conv_learner import *

learn = ConvLearner.from_model_data(model, train_iter)
learn.crit = F.cross_entropy

In [ ]:
learn.lr_find(wds=1e-4, end_lr=100)

In [ ]:
learn.sched.plot(10, 1)